<a href="https://colab.research.google.com/github/caomy7/PLSP/blob/main/02_GEE_Steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=SSc2Twukf-y3RBp5l0hxqwydF21rIh-G0sbAP9YxIhM&tc=8sliQv8h3jFEWsuB0s0-BJv-4XDTFHuKDag2frx61rs&cc=wL3fz612GsMZrLbJmS4zC-_b-OlHWeb_qdEd3OwG07A

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdf_xgSTkNP2q6-tUHx_wZyKceb_K6Uu02Yx60IwyL9bi814jS2Ma0E

Successfully saved authorization token.


In [52]:
#@title
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **加载Sentinel2数据，并除云**

In [75]:
import ee
import geemap
# import oeel

Map = geemap.Map()
# Map

vegIndex = 'EVI2' 
th = 0.2 
threshMin = 0.1
scale = 50

year1 = "2020" 
lag = 20 
startDate = year1+'-01-01'
endDate = year1+'-12-31'

print(startDate,endDate)


geom =ee.Geometry.Point([17.07398, 67.7578]);
point = geom 
roi = point.buffer(100000).bounds() 

# roi = ee.Geometry.Polygon([[-97.72,31.61],[-82.85,31.61],[-82.85,37.67],[-97.72,37.61],[-97.72,31.61]])



CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1


dataset = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.date(startDate, endDate)).filterBounds(roi)
# dataset = ee.ImageCollection("COPERNICUS/S2_SR")
size = dataset.size()
print(size.getInfo())

def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).select("B.*").copyProperties(image,["system:time_start"])


filtered = dataset.map(maskS2clouds)

rgbVis = {
  "bands": ['B11', 'B8', 'B3'],
  "min": 0,
  "max": 3000
};

Map.addLayer(filtered,rgbVis,"sentinel_cloudfree")
Map
Map.addLayerControl()
Map

2020-01-01 2020-12-31
2502


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# **显示EVI2数据**

In [76]:

def EVI2(image):
  return image.expression(  
    '2.5*(NIR-RED) / (NIR+2.4*RED+1)',{  
      'RED': image.select('B4'),  
      'NIR': image.select('B8'),  
    })

  # return image.addBands(evi2.rename('EVI2'));

withEVI2 = filtered.map(EVI2);
size = withEVI2.size()
print(size.getInfo())

collection = withEVI2.select('EVI2');

Map = geemap.Map()
# Map
vis = {"min": 0, "max": 1, "palette": [
  'FFFFFF', 'CE7E45', 'FCD163', '66A000', '207401',
  '056201', '004C00', '023B01', '012E01', '011301'
]};

Map.addLayer(withEVI2,vis,"EVI2")
Map
Map.addLayerControl()
Map

# geemap.ee_export_image_collection_to_drive(withEVI2, folder='export', scale=10)
# # geemap.ee_export_image_to_drive(withEVI2,description='evi2', folder='export', scale=10)

# print("finished")

2502


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# **GEE**

In [93]:

import ee
import geemap
# import oeel

Map = geemap.Map()
# Map

# SECTION 1 - Define parameters and setup

vegIndex = 'evi2' 
th = 0.2 
threshMin = 0.1
scale = 50

year1 = "2020" 
lag = 20 
startDate = year1+'-01-01'
endDate = year1+'-12-31'

print(startDate,endDate)


geom =ee.Geometry.Point([17.07398, 67.7578]);
point = geom 
roi = point.buffer(100000).bounds() 

# SECTION 3 - Prepare Sentinel-2 data 

# roi = ee.Geometry.Polygon([[-97.72,31.61],[-82.85,31.61],[-82.85,37.67],[-97.72,37.61],[-97.72,31.61]])
S2 = ee.ImageCollection("COPERNICUS/S2_SR").filterDate(startDate,endDate).filterBounds(roi).filterMetadata('CLOUD_COVERAGE_ASSESSMENT','less_than',70).filterMetadata('SNOW_ICE_PERCENTAGE','less_than',90);


# Mask non-valid observations and reclassify snow values

def mask_no(im):
  SCL = im.select('SCL')
  SCLmask = SCL.eq(1).Or(SCL.eq(4)).Or(SCL.eq(5)).Or(SCL.eq(11)) #mask for non-valid observations
  snowMask = SCL.eq(11) #snow 
    
  blue = im.select('B2').multiply(0.0001)
  green = im.select('B3').multiply(0.0001)
  red = im.select('B4').multiply(0.0001)
  nir = im.select('B8').multiply(0.0001)
  swir = im.select('B12').multiply(0.0001)
    
  ndvi = im.normalizedDifference(['B8','B4']).rename('ndvi')
  evi = (ee.Image(2.5).multiply(nir.subtract(red))).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)).rename('evi')
    
  evi2 = (ee.Image(2.5).multiply(nir.subtract(red))).divide(nir.add(red.multiply(2.4)).add(1)).rename('evi2')
    
  gcc = green.divide(green.add(red).add(blue)).rename('gcc')
  alpha = ee.Image(0.51) # alpha parameter in NDPI formula
  ndpi = (nir.subtract(alpha.multiply(red).add((ee.Image(1).subtract(alpha)).multiply(swir)))).divide(nir.add(alpha.multiply(red).add((ee.Image(1).subtract(alpha)).multiply(swir)))).rename('ndpi')
    
    #  select VI of interest
  bio = ndvi.addBands(evi2).addBands(evi).addBands(ndpi).addBands(gcc).select(vegIndex).rename('bio') 

  return bio.where(bio.lt(threshMin),threshMin).where(snowMask,threshMin).updateMask(SCLmask).set('system:time_start', im.get('system:time_start'))

S2 = S2.map(mask_no)
data=S2.size()
print(data.getInfo())


2020-01-01 2020-12-31
1403


In [ ]:
# SECTION 4 - Generate composites

# listDates = ee.List.sequence(ee.Date(startDate).millis(),ee.Date(endDate).millis(),86400000*lag)
listDates = ee.List.sequence(ee.Date(startDate).millis(),ee.Date(endDate).millis(),86400000*lag)
# print(listDates)

#  map the dates

def dds(dd):
  date_window = ee.Date(ee.Number(dd)) 
  date_startW = date_window.advance(-lag/2, 'days') 
  date_endW = date_window.advance(lag/2, 'days') 
  col_window = S2.filterDate(date_startW,date_endW) 
  out = col_window.reduce(ee.Reducer.mean())
  r = out.set('system:time_start',date_window.millis()).set('empty', col_window.size().eq(0))
 
  # return r
  return out.set('system:time_start',date_window.millis()).set('empty', col_window.size().eq(0))

colDekadsRaw = ee.ImageCollection(listDates.map(dds))


colDekadsRaw = colDekadsRaw.flatten()
# colDekadsRaw

colDekadsEmpty = colDekadsRaw.filterMetadata('empty', 'equals', 1).map(lambda im:ee.Image(threshMin).rename('bio_mean').copyProperties(im,['system:time_start']))

colDekads = colDekadsRaw.filterMetadata('empty', 'equals', 0).merge(colDekadsEmpty)
print(colDekads)
# size = colDekads.size()
# print(size.getInfo())


In [95]:

def i(im):
  date_window = ee.Date(im.get('system:time_start'))
  date_startW = date_window.advance(-lag*2, 'days')
  date_endW = date_window.advance(lag*2, 'days')
  
  
  meanIm1 = ee.Image(colDekads.filterDate(date_startW,date_window.advance(1, 'days'))).reduce(ee.Reducer.mean())
  meanIm2 = ee.Image(colDekads.filterDate(date_window.advance(-1, 'days'),date_endW)).reduce(ee.Reducer.mean())
  
  meanIm = (meanIm1.add(meanIm2)).divide(2)
  
  return ee.Image(im).unmask(meanIm).copyProperties(im,['system:time_start'])

colDekads = colDekads.map(i)
colDekads = colDekads.sort('system:time_start')
print(colDekads.getInfo())
# Map.addLayer(colDekads,vis,"colDekads")
# Map
# Map.addLayerControl()
# Map



EEException: ignored

In [96]:
# SECTION 2 - Define function for cubic interpolation

def divide_fun(x,diff12,a,b,c,d,xDates):
  im = ee.Image(ee.Number(x).divide(diff12));
  return (im.pow(3)).multiply(a).add((im.pow(2)).multiply(b)).add(im.multiply(c)).add(d).set('system:time_start',ee.Number(ee.Date(xDates).get(x)));

def ii(ii,step):
  ii = ee.Number(ii);
  p0 = ee.Image(collection.toList(10000).get(ee.Number(ii).subtract(1)));
  p1 = ee.Image(collection.toList(10000).get(ii));
  p2 = ee.Image(collection.toList(10000).get(ee.Number(ii).add(1)));
  p3 = ee.Image(collection.toList(10000).get(ee.Number(ii).add(2)));
  
  diff01 = ee.Date(p1.get('system:time_start')).difference(ee.Date(p0.get('system:time_start')), 'day');
  diff12 = ee.Date(p2.get('system:time_start')).difference(ee.Date(p1.get('system:time_start')), 'day');
  diff23 = ee.Date(p3.get('system:time_start')).difference(ee.Date(p2.get('system:time_start')), 'day');
    
  diff01nor = diff01.divide(diff12);
  diff12nor = diff12.divide(diff12);
  diff23nor = diff23.divide(diff12);
    
  f0 = p1;
  f1 = p2;
  f0p = (p2.subtract(p0)).divide(diff01nor.add(diff12nor));
  f1p = (p3.subtract(p1)).divide(diff12nor.add(diff23nor));
  
  a = (f0.multiply(2)).subtract(f1.multiply(2)).add(f0p).add(f1p);
  b = (f0.multiply(-3)).add(f1.multiply(3)).subtract(f0p.multiply(2)).subtract(f1p);
  c = f0p;
  d = f0;
    
  xValues = ee.List.sequence(0,diff12.subtract(1),step); 
  xDates = ee.List.sequence(p1.get('system:time_start'),p2.get('system:time_start'),86400000);
  
  interp = (xValues.map(divide_fun));
  
  return interp 

def cubicInterpolation(collection,step):
  listDekads = ee.List.sequence(1,collection.size().subtract(3),1);
  colInterp = listDekads.map(ii)

  colInterp = ee.ImageCollection(colInterp.flatten());
  
  return colInterp


# SECTION 6 - Interpolate time series

interp = cubicInterpolation(colDekads,1)
print(interp)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# SECTION 7 - Estimate phenology metrics

init = ee.Image(ee.Date(str((int(year1)-1))+'-12-31').millis());

def EVI2_interp(im):
  return im.rename('bio_interp').addBands(im.metadata('system:time_start','date1')).set('system:time_start', im.get('system:time_start'))

minND = ee.Image(threshMin)
maxND = max(colDekads)
# maxND = colDekads.max()
amplitude = maxND.subtract(minND)

thresh = amplitude.multiply(th).add(minND).rename('bio_interp')

def th_interp(im):
  out = im.select('bio_interp').gt(thresh);
  return im.updateMask(out).copyProperties(im,['system:time_start'])


col_aboveThresh = interp.map(th_interp)
print(col_aboveThresh)

SoS = col_aboveThresh.reduce(ee.Reducer.firstNonNull()).select('date1_first').rename('SoS')
SoS_doy = SoS.subtract(init).divide(86400000);

EoS = col_aboveThresh.reduce(ee.Reducer.lastNonNull()).select('date1_last').rename('EoS')
EoS_doy = EoS.subtract(init).divide(86400000); 

phenoPalette = ['ff0000','ff8d00','fbff00','4aff00','00ffe7','01b8ff','0036ff','fb00ff']
visSoS = {"min":120,"max":200,"palette":phenoPalette}
visEoS = {"min":180,max:300,"palette":phenoPalette}
Map.addLayer(SoS_doy,visSoS,'SoS',true)
Map.addLayer(EoS_doy,visEoS,'EoS',false)
Map